# Clastering. Part 2

---
Author: Anatoliy Durkin

Updated: 02.05.2025

---
В ноутбуке рассмотерны методы кластеризации

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
peng = pd.read_csv('penguins.csv')
peng = peng[peng['sex']!='.']
peng = peng[(peng['flipper_length_mm']<1000) & (peng['flipper_length_mm']>100)]
peng = peng.dropna().reset_index(drop=True)
peng = pd.get_dummies(peng, drop_first=True)

peng_norm = pd.DataFrame(StandardScaler().fit_transform(peng), columns=peng.columns).dropna()


cust = pd.read_csv('customer_segmentation.csv')
cust = cust.drop(['ID', 'Dt_Customer'], axis=1)
cust['Education'] = LabelEncoder().fit_transform(cust['Education'])
cust['Marital_Status'] = LabelEncoder().fit_transform(cust['Marital_Status'])

cust_norm = pd.DataFrame(StandardScaler().fit_transform(cust), columns=cust.columns).dropna()
cust_pca = PCA(n_components=2).fit_transform(cust_norm)

In [ ]:
peng_norm.head()

In [ ]:
cust_norm.head()

## Gaussian Mixture Models (GMM)

Когда использовать:

- Кластеры с перекрытием
- Нужны вероятности принадлежности
- Разные размеры/формы кластеров

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs

In [ ]:
# Данные с разной дисперсией
X, _ = make_blobs(n_samples=300, centers=3, cluster_std=[1.0, 2.5, 0.5], random_state=42)

In [ ]:
plt.scatter(X[:,0], X[:,1])
plt.title("Данные с разной дисперсией")
plt.show()

In [ ]:
gmm = GaussianMixture(n_components=3)
gmm.fit(X)
clusters = gmm.predict(X)

In [ ]:
# Границы кластеров
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
Z = gmm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

In [ ]:
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X[:, 0], X[:, 1], c=clusters, s=50)
plt.title("GMM с разными дисперсиями")
plt.show()

In [ ]:
gmm = GaussianMixture(n_components=3, covariance_type='spherical')
gmm.fit(X)
clusters = gmm.predict(X)

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
Z = gmm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X[:, 0], X[:, 1], c=clusters, s=50)
plt.title("GMM с разными дисперсиями")
plt.show()

In [ ]:
silhouette = []
for k in range(2, 7):
    clusters = KMeans(n_clusters=k, random_state=42).fit_predict(X)
    silhouette.append(silhouette_score(X, clusters))

plt.plot(range(2, 7), silhouette, marker='o')
plt.xlabel("Число кластеров")
plt.ylabel("Silhouette")
plt.title("График силуэт")
plt.show()

In [ ]:
silhouette = []
for k in range(2, 7):
    clusters = GaussianMixture(n_components=3).fit_predict(X)
    silhouette.append(silhouette_score(X, clusters))

plt.plot(range(2, 7), silhouette, marker='o')
plt.xlabel("Число кластеров")
plt.ylabel("Silhouette")
plt.title("График силуэт")
plt.show()

Также для выбора количества кластеров можно использовать BIC, но это подходит только для GMM.

In [ ]:
bics = []
for n in range(1, 10):
    gmm = GaussianMixture(n_components=n).fit(X)
    bics.append(gmm.bic(X))
plt.plot(range(1, 10), bics)
plt.xlabel("Число кластеров")
plt.title("График BIC")
plt.show()

Воспользуемся этим методом для наших наборов данных. Также определите, какое число кластеров оптимально по BIC для каждого набора.

In [ ]:
# Ваш код
...

In [ ]:
gmm = GaussianMixture(n_components=3)
gmm.fit(peng_norm.drop(['sex_MALE'], axis=1))
clusters = gmm.predict(peng_norm.drop(['sex_MALE'], axis=1))

plt.scatter(peng['culmen_length_mm'], peng['culmen_depth_mm'], c=clusters, s=50)
plt.title("GMM с разными дисперсиями")
plt.show()

In [ ]:
# Ваш код
...

In [ ]:
gmm = GaussianMixture(n_components=3)
gmm.fit(cust_norm)
clusters = gmm.predict(cust_norm)

plt.scatter(cust_pca[:, 0], cust_pca[:, 1], c=clusters, s=50)
plt.title("GMM с разными дисперсиями")
plt.show()

## Spectral Clustering

Когда использовать:

- Кластеры сложной формы (полумесяцы, кольца)
- Когда DBSCAN не справляется

In [ ]:
from sklearn.datasets import make_moons
from sklearn.cluster import SpectralClustering

In [ ]:
X, _ = make_moons(n_samples=300, noise=0.05, random_state=42)

plt.scatter(X[:,0], X[:,1])
plt.title("Данные полумесяцы")
plt.show()

In [ ]:
spec = SpectralClustering(n_clusters=2, affinity='nearest_neighbors')
clusters = spec.fit_predict(X)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis')
plt.title("Spectral Clustering: Полумесяцы")
plt.show()

Сравнение работы двух методов

In [ ]:
from sklearn.cluster import DBSCAN

X, _ = make_moons(n_samples=300, noise=0.09, random_state=42)

spec = SpectralClustering(n_clusters=2, affinity='nearest_neighbors')
clusters = spec.fit_predict(X)

dbscan = DBSCAN(eps=0.3, min_samples=5)
clusters_db = dbscan.fit_predict(X)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis')
ax1.set_title("Spectral Clustering")
ax2.scatter(X[:, 0], X[:, 1], c=clusters_db, cmap='viridis')
ax2.set_title("DBSCAN")
plt.show()

Что получится сделать с нашими датасетами?

In [ ]:
# Ваш код
...

## OPTICS (расширенный DBSCAN)

Когда использовать:

- Кластеры разной плотности
- Автоматический подбор параметров

Отличие от DBSCAN:

- Не требует точного задания eps
- Строит reachability-plot

Как работает алгоритм
- Строит «упорядоченный» список точек, где близкие в списке точки геометрически близки в данных.
- Для каждой точки вычисляет «достижимое расстояние» — минимальный радиус, включающий min_samples соседей.
- Кластеры — это «впадины» на графике достижимости (reachability plot).

In [ ]:
from sklearn.cluster import OPTICS
X, _ = make_blobs(n_samples=300, centers=3, cluster_std=[0.5, 2.0, 0.8], random_state=42)

clust = OPTICS(min_samples=10, xi=0.05)
clust.fit(X)

# Reachability plot
plt.figure(figsize=(10, 4))
plt.plot(clust.reachability_[clust.ordering_], 'o-')
plt.title("Reachability Plot")
plt.show()

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=clust.labels_, cmap='viridis')
plt.title("OPTICS")
plt.show()

И вновь обратимся к нашим датасетам.

In [ ]:
# Ваш код
...

## Выбор метода кластеризации

https://scikit-learn.ru/stable/modules/clustering.html

## Интерпретация кластеров

Кластеризация — это не конец анализа, а начало. Найдя группы, мы должны понять:

- Что их объединяет? (Какие признаки значимы)
- Как их можно назвать? (Смысловая интерпретация)
- Как использовать это в бизнесе/науке?

### Анализ центроидов (для K-Means, GMM)

Как работает:

- Центроид — это усреднённая точка кластера в пространстве признаков.
- Сравниваем значения признаков центроидов между кластерами.

In [ ]:
gmm = GaussianMixture(n_components=3)
gmm.fit(peng_norm.drop(['sex_MALE'], axis=1))
clusters = gmm.predict(peng_norm.drop(['sex_MALE'], axis=1))

peng['Кластер'] = clusters

# Средние значения по кластерам
peng.groupby('Кластер').mean()

Выберите метод кластеризации для второго набора данных и проанализируйте центроиды.

In [ ]:
# Ваш код
...

Важность признаков.

ANOVA — определяет, какие признаки значимо различаются между кластерами. Возвращает значение F-статистики.

In [ ]:
from sklearn.feature_selection import f_oneway

In [ ]:
f_values, p_values = f_oneway(peng[clusters==0], peng[clusters==1], peng[clusters==2])
print(f_values)
print(p_values)

SHAP для кластеризации — показывает вклад каждого признака в кластер.

In [ ]:
import shap

In [ ]:
gmm = GaussianMixture(n_components=3).fit(peng_norm.drop(['sex_MALE'], axis=1))
explainer = shap.KernelExplainer(gmm.predict, peng_norm.drop(['sex_MALE'], axis=1))
shap_values = explainer.shap_values(peng_norm.drop(['sex_MALE'], axis=1))
shap_values

### Визуализация

Parallel Coordinates — это как "профиль" каждого кластера. Линии, которые идут отдельно от других, — это ключевые различия.

In [ ]:
gmm = GaussianMixture(n_components=3)
gmm.fit(peng_norm.drop(['sex_MALE'], axis=1))
clusters = gmm.predict(peng_norm.drop(['sex_MALE'], axis=1))

peng_norm['Кластер'] = clusters
pd.plotting.parallel_coordinates(peng_norm, 'Кластер')